In [ ]:
import os
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

In [ ]:
import torch
print(torch.__version__)

In [ ]:
import gymnasium as gym
from stable_baselines3 import PPO
from stable_baselines3.common.callbacks import BaseCallback
import numpy as np

# 사용자 정의 모듈 Import
from environment_src.heater_gym_env import HeaterEnv  # (이전 단계에서 만든 환경 파일)
from environment_src.animation import TrainingAnimator # (위에서 만든 애니메이션 파일)

print("모듈 로드 완료.")

In [ ]:
class TrainingLogCallback(BaseCallback):
    """
    학습 도중 주기적으로 성능을 테스트하고 데이터를 저장하는 콜백
    """
    def __init__(self, check_freq, target_temp=50.0, verbose=1):
        super(TrainingLogCallback, self).__init__(verbose)
        self.check_freq = check_freq
        self.target_temp = target_temp
        # 시뮬레이션 기록을 저장할 리스트
        self.records = []

        # 테스트용 별도 환경 (학습 환경에 영향을 주지 않기 위해)
        self.eval_env = HeaterEnv(target_temp=self.target_temp)

    def _on_step(self) -> bool:
        # check_freq 스텝마다 실행
        if self.n_calls % self.check_freq == 0:

            # 1. 테스트 시뮬레이션 수행
            obs, _ = self.eval_env.reset()
            times, temps, outputs, rewards = [], [], [], []

            for t in range(600): # 600초(10분) 시뮬레이션
                # 현재 모델로 행동 예측 (deterministic=True: 확률적 요소 배제)
                action, _ = self.model.predict(obs, deterministic=True)
                obs, r, done, _, _ = self.eval_env.step(action)

                # 기록
                rewards.append(r)
                times.append(t)
                temps.append(obs[1]) # 온도
                outputs.append(self.eval_env.prev_output) # 출력

            # 2. 결과 저장
            record = {
                'step': self.num_timesteps, # 현재 총 학습 스텝 수
                'times': times,
                'temps': temps,
                'outputs': outputs,
                'rewards': rewards
            }
            self.records.append(record)

            if self.verbose > 0:
                last_temp = temps[-1]
                print(f"Step {self.num_timesteps}: Test run finished. Final Temp: {last_temp:.2f}°C")

        return True

In [ ]:
# --- [Cell 3] 매 에피소드마다 기록하며 학습 실행 ---

# 1. 환경 및 모델 생성
env = HeaterEnv(target_temp=50.0)
model = PPO("MlpPolicy", env, verbose=0, learning_rate=0.0003)

# 2. 콜백 설정 (핵심 변경 사항!)
# check_freq를 환경의 max_steps(600)와 동일하게 설정합니다.
# 이렇게 하면 매 에피소드(실험)가 끝날 때마다 현재 지능으로 테스트를 수행하고 기록합니다.
EPISODE_LENGTH = 600
EPOCH = 30000 # 총 50번의 에피소드(시행착오)를 경험함

print(f"--- 학습 시작 (총 {EPOCH // EPISODE_LENGTH}번의 에피소드) ---")
callback = TrainingLogCallback(check_freq=EPISODE_LENGTH, target_temp=50.0)


model.learn(total_timesteps=EPOCH, callback=callback)
print("--- 학습 완료 ---")

In [ ]:
# --- [Cell 4] 시행착오 과정 애니메이션 (매 에피소드 반영) ---
print(f"수집된 데이터 프레임 수: {len(callback.records)}개")
print("애니메이션 생성 중... (데이터가 많아 시간이 조금 걸릴 수 있습니다)")

# 1. 애니메이터 객체 생성
animator = TrainingAnimator(callback.records, target_temp=50.0)

# 2. 모핑 비디오 생성 (파라미터 조정)
# frames_per_log=5: 데이터가 촘촘하므로 보간 프레임을 줄여도 부드럽습니다.
# interval=30: 재생 속도를 약간 빠르게 하여 전체 과정을 지루하지 않게 봅니다.
video = animator.create_morphing_video(frames_per_log=5, interval=30)

# 3. 출력
video